In [1]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [2]:
import os
import random
import warnings

import numpy as np
import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base

warnings.filterwarnings("ignore")

seed = 46
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)

2024-06-04 02:59:12.495810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 02:59:12.843634: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 02:59:14.081193: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

In [3]:
import bz2
import gc
import hashlib
import json
import math
import multiprocessing
import random
import time
from glob import glob
from typing import List, Set

import lz4.frame
import matplotlib.pyplot as plt
import numpy as np
import plyvel
import polars as pl
from penguinml.utils.timer import Timer
from tqdm import tqdm

import whoosh_utils
from const import ALL_KEYS, CPC2TOKENS_PATH, INF, KEY2QUERY, NUM_CPU
from solver import SimulatedAnnealing
from utils import (
    calc_bytes,
    compute_ap,
    evaluate,
    load_list_bz2,
    read_bytes_in_range,
    save_list_bz2,
)


def stable_hash(obj, mod: int = 20):
    """
    NOTE: objは`json.dumps`する前のオブジェクト
    """
    obj_str = json.dumps(obj, sort_keys=True)
    hash_bytes = hashlib.sha256(obj_str.encode()).digest()
    hash_int = int.from_bytes(hash_bytes, byteorder="big")
    return hash_int % mod

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
N_SPLIT = 10

for i in range(N_SPLIT):
    !rm -rf tokenized-db-{i}

In [5]:
tokenized_files = glob("../data/*.bz2") + glob("../data2/*.bz2")
len(tokenized_files)

13307660

In [6]:
# データベースの作成
for i in range(N_SPLIT):
    os.makedirs(f"tokenized-db-{i}", exist_ok=True)
dbs = [plyvel.DB(f"tokenized-db-{i}/db", create_if_missing=True) for i in range(N_SPLIT)]
txts = [open(f"tokenized-db-{i}/index.lz4", "wb") for i in range(N_SPLIT)]

cursors = [0] * N_SPLIT
debug_center2txt = {}
debug_count = 0
total_unique = 0
total_dup = 0

pbar = tqdm(tokenized_files)
for file in pbar:
    exist = set()
    try:
        data = load_list_bz2(file)
    except:
        print(f"{file} is broken")
        continue
    center = os.path.basename(file).split(".")[0]

    # dict -> str
    txt = json.dumps(data)

    split = stable_hash(center, mod=N_SPLIT)
    start = cursors[split]
    compressed_chunk = lz4.frame.compress(txt.encode("utf-8"))
    txts[split].write(compressed_chunk)
    cursors[split] += len(compressed_chunk)
    end = cursors[split]

    dbs[split].put(center.encode(), json.dumps((start, end)).encode())

    debug_count += 1
    if debug_count % 10000 == 0:
        debug_center2txt[center] = txt

    # if debug_count > 100000:
    #     break

 80%|███████▉  | 10589091/13307660 [5:33:38<1:15:15, 602.04it/s]

../data/US-2009572-A.json.bz2 is broken


100%|██████████| 13307660/13307660 [7:05:22<00:00, 521.40it/s]   


In [7]:
def read_lz4_in_range(file_path, start_byte, end_byte):
    with open(file_path, "rb") as f:
        f.seek(start_byte)
        compressed_content = f.read(end_byte - start_byte)
        content = lz4.frame.decompress(compressed_content)
        return content.decode("utf-8")

In [8]:
for center, txt in tqdm(debug_center2txt.items()):
    split = stable_hash(center, mod=N_SPLIT)
    start, end = json.loads(dbs[split].get(center.encode()))
    read_txt = read_lz4_in_range(f"tokenized-db-{split}/index.lz4", start, end)
    assert txt == read_txt
json.loads(read_txt)

100%|██████████| 1330/1330 [00:01<00:00, 863.75it/s]


{'title': ['magnetohydrodynamic', 'generator'],
 'abstract': [],
 'claims': ['magnetohydrodynamic',
  'generator',
  'comprising',
  'elongated',
  'plasma',
  'generating',
  'means',
  'magnetic',
  'means',
  'producing',
  'magnetic',
  'field',
  'transverse',
  'length',
  'plasma',
  'generating',
  'means',
  'said',
  'plasma',
  'generating',
  'means',
  'including',
  'opposite',
  'lateral',
  'elongated',
  'walls',
  'having',
  'opposing',
  'electrode',
  'pairs',
  'displaceable',
  'toward',
  'each',
  'other',
  'compensating',
  'wear',
  'said',
  'electrodes',
  'photoelectric',
  'system',
  'extending',
  'along',
  'interior',
  'said',
  'plasma',
  'generating',
  'means',
  'and',
  'defining',
  'datum',
  'position',
  'electrodes',
  'electrode',
  'drive',
  'means',
  'each',
  'electrode',
  'and',
  'regulating',
  'device',
  'responsive',
  'photoelectric',
  'system',
  'and',
  'connected',
  'said',
  'respective',
  'drive',
  'means',
  'peri